In [104]:
import numpy as np
import pandas as pd

from datetime import date

from statsmodels.tsa.stattools import coint

In [105]:
data_file=pd.read_csv('C:/Users/billc/Career/Stock_strategies/Pair_trades/Output_files/stock_database.csv')

In [106]:
data_file['Date']=data_file['Date'].astype('datetime64[ns]')
data_file.set_index('Date', inplace=True)
data_file.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,20.579165,4.496876,39.094307,6.604801,NaN,22.606730,18.022673,33.464951,37.090000,24.001997,...,NaN,19.886999,47.467609,32.957073,18.717176,NaN,20.419832,55.590977,11.827960,NaN
2010-01-05,20.355618,5.005957,38.861965,6.616219,NaN,22.445440,17.877064,33.671783,37.700001,23.964100,...,NaN,19.636250,47.652931,32.565281,18.738873,NaN,20.349995,57.350761,12.244997,NaN
2010-01-06,20.283297,4.798555,39.200817,6.510980,NaN,22.233210,17.976341,34.029732,37.619999,23.918631,...,NaN,19.503048,48.064804,32.779831,18.565357,NaN,20.204515,57.332245,13.309779,NaN
2010-01-07,20.257004,4.939965,39.191139,6.498945,NaN,21.876654,18.125263,33.997910,36.889999,23.729166,...,NaN,19.307158,47.913792,33.208939,18.652111,NaN,20.198702,58.647461,14.800472,NaN
2010-01-08,20.250422,4.845690,39.346043,6.542150,NaN,22.114357,18.217920,33.862690,36.689999,23.865580,...,NaN,19.589245,47.721592,33.208939,18.587051,NaN,20.204515,57.415607,14.560900,NaN


In [107]:
#pair=data_file.iloc[:,[1,2]]
#pair.dropna(how='any', axis=0, inplace=True)
#tt=coint(pair.iloc[:,0], pair.iloc[:,1])

In [108]:
test_results=pd.DataFrame(columns=('t_stat', 'p_value', 'CI_1', 'CI_5','CI_10', 'i', 'j'))
temp_list=[]
main_list=[]

stock_list=data_file.columns.to_list()
for i in range(0, len(stock_list)-1):
    for j in range(i+1, len(stock_list)):
        pair=data_file.iloc[:,[i,j]]
        pair.dropna(how='any', axis=0, inplace=True)
        
        #print(pair.isnull().sum())
        result=coint(pair.iloc[:,0], pair.iloc[:,1])
        temp_list=[result[0], result[1], result[2][0], result[2][1], result[2][2], i, j]
        main_list.append(temp_list)

test_results=pd.DataFrame(main_list)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
test_results.head()